(unrelated links)
- https://www.youtube.com/watch?v=ddCO2714W-o
- https://www.youtube.com/watch?v=UdADuHJUX6Q

In [68]:
import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.keras.layers import Dense
import numpy as np
import matplotlib.pyplot as plt

In [69]:
def prepare_mnist_data(ds):
  #Uint8 to float32
  ds = ds.map(lambda img, target: (tf.cast(img, tf.float32), target))
  #Flatten images into vectors
  ds = ds.map(lambda img, target: (tf.reshape(img, (-1,)), target))
  #Sloppy input normalization, [0, 255] to [-1, 1] (standart normal (gaussian) distribution)
  ds = ds.map(lambda img, target: ((img/128.)-1., target))
  #One-hot targets 
  ds = ds.map(lambda img, target: (img, tf.one_hot(target, depth=10)))
  #Cache progress ("deterministic"?)
  ds = ds.cache()
  #Shuffle, batch, prefetch
  ds = ds.shuffle(1000, seed=42)
  ds = ds.batch(32)
  ds = ds.prefetch(20)
  #return preprocessed dataset
  return ds

(train_ds, test_ds), ds_info = tfds.load("mnist", split=['train', 'test'], as_supervised=True, with_info=True)

# tfds.show_examples(train_ds, ds_info)
# print(ds_info)

train_ds = train_ds.apply(prepare_mnist_data)
test_ds = test_ds.apply(prepare_mnist_data);

Answers :
- Training/test images : 6,000 / 10,000
- Image shape : (28, 28,)
- Range of pixel values : (0, 255)

(unrelated links) 
- https://www.youtube.com/watch?v=upsAvKvdROY

In [76]:
from tensorflow.keras.layers import Dense
import numpy as np

#Fully connected feed-forward dnn to classify MNIST images using ’Dense’ layers
class MyNetwork(tf.keras.Model):

  def __init__(self, hidden = np.array([256, 256]), activations = None):
    super(MyNetwork, self).__init__()

    if (activations == None):
      activations = np.array(["relu" for i in range(len(hidden))])
      #Todo : Error if len(size) != len(activations)

    self.sizes = np.append(hidden, np.array([10]))
    self.activations = np.append(np.array([tf.keras.activations.get(activations[i]) for i in range(len(activations))]), tf.nn.softmax)
    #(Don't use "layers" variable name -> conflicting namespace)
    self.dense_layers = [tf.keras.layers.Dense(self.sizes[i], self.activations[i]) for i in range(len(self.sizes))]

  @tf.function
  def call(self, inputs):
    for i in range(len(self.dense_layers)):
      inputs = self.dense_layers[i](inputs)
    return inputs

In [77]:
def train_step(model, input, target, loss_function, optimizer = "adam"):
  # loss_object and optimizer_object are instances of respective tensorflow classes
  with tf.GradientTape() as tape:
    prediction = model(input)
    loss = loss_function(target, prediction)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))
  return loss

In [78]:
def test(model, test_data, loss_function):
  # test over complete test data

  test_accuracy_aggregator = []
  test_loss_aggregator = []

  for (input, target) in test_data:
    prediction = model(input)
    sample_test_loss = loss_function(target, prediction)
    sample_test_accuracy =  np.argmax(target, axis=1) == np.argmax(prediction, axis=1)
    sample_test_accuracy = np.mean(sample_test_accuracy)
    test_loss_aggregator.append(sample_test_loss.numpy())
    test_accuracy_aggregator.append(np.mean(sample_test_accuracy))

  test_loss = tf.reduce_mean(test_loss_aggregator)
  test_accuracy = tf.reduce_mean(test_accuracy_aggregator)

  return test_loss, test_accuracy

In [82]:
def train_mynet(epochs, model, loss, optimizer):

  train_losses = []
  test_losses = []
  test_accuracies = []
  # train_accuracies = []
  #testing once before we begin
  test_loss, test_accuracy = test(model, test_ds, loss)
  test_losses.append(test_loss)
  test_accuracies.append(test_accuracy)
  #check how model performs on train data once before we begin
  train_loss, train_accuracy = test(model, train_ds, loss)
  train_losses.append(train_loss)
  # train_accuracies.append(train_accuracy)


  # We train number of epochs times
  for epoch in range(epochs):
    print(f'Epoch: {str(epoch)} starting with accuracy {test_accuracies[-1]}')

    #training (and checking in with training)
    epoch_loss_agg = []
    for input,target in train_ds:
      train_loss = train_step(model, input, target, loss, optimizer)
      epoch_loss_agg.append(train_loss)
    
    #track training loss
    train_losses.append(tf.reduce_mean(epoch_loss_agg))

    #testing, so we can track accuracy and test loss
    test_loss, test_accuracy = test(model, test_ds, loss)
    test_losses.append(test_loss)
    test_accuracies.append(test_accuracy)

  return(train_losses, test_losses, test_accuracies)

In [87]:
import matplotlib.pyplot as plt

def visualization(train_losses, test_losses, test_accuracies, train_accuracies = None):
  '''
  Visualizes accuracy and loss for training and test data using the mean of each epoch.
  Loss is displayed in a regular line , accuracy in a dotted line.
  Training data is displayed in blue , test data in red.
  Parameters
  ----------
  train_losses : numpy . ndarray
  training losses
  train_accuracies : numpy . ndarray
  training accuracies
  test_losses : numpy . ndarray
  test losses
  test_accuracies : numpy . ndarray
  test accuracies
  '''
  plt.figure ()
  line1 = plt.plot(train_losses, "b-")
  line2 = plt.plot(test_losses, "r-")
  # line3 = plt.plot(train_accuracies, "b:")
  line4 = plt.plot(test_accuracies, "r:")
  plt.xlabel("Training steps")
  plt.ylabel("Loss / Accuracy")
  # plt.legend((line1, line2, line3, line4), ("training loss", "test loss", "train accuracy", "test accuracy"))
  plt.legend((line1, line2, line4), ("training loss", "test loss", "test accuracy"))
  plt.show()

In [83]:
#MAIN---------------------------------------------------------------------------
n_epochs = 10

hidden_architecture = [256, 256]
hidden_activation_functions = ["relu", "relu"]
model = MyNetwork(hidden_architecture, hidden_activation_functions)

cross_entropy_loss = tf.keras.losses.CategoricalCrossentropy()
#...
loss_function = cross_entropy_loss

learning_rate = 0.001
optimizer = tf.keras.optimizers.SGD(learning_rate)

In [85]:
train_losses, test_losses, test_accuracies = train_mynet(n_epochs, model, loss_function, optimizer)
visualization(train_losses, test_losses, test_accuracies)

Epoch: 0 starting with accuracy 0.9048522364217252
Epoch: 1 starting with accuracy 0.9093450479233227
Epoch: 2 starting with accuracy 0.9139376996805112
Epoch: 3 starting with accuracy 0.9174321086261981
Epoch: 4 starting with accuracy 0.9218250798722045
Epoch: 5 starting with accuracy 0.9225239616613419
Epoch: 6 starting with accuracy 0.9269169329073482
Epoch: 7 starting with accuracy 0.9288138977635783
Epoch: 8 starting with accuracy 0.9303115015974441
Epoch: 9 starting with accuracy 0.9330071884984026
